In [1]:
import cv2

In [2]:
import numpy as np

In [3]:
import pyautogui

In [4]:
py = cv2.imread("C:/Users/DELL/Desktop/CV2_course_material-20190822T085446Z-001/CV2_course_material/py.jpg")

In [5]:
gray = cv2.cvtColor(py,cv2.COLOR_BGR2GRAY)

contours,_ = cv2.findContours(gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

cnts = max(contours[1:], key=cv2.contourArea)
hull = cv2.convexHull(cnts)

cv2.drawContours(py,cnts,-1,(255,0,0),3)
cv2.drawContours(py,[hull],-1,(255,255,0),3)

cv2.imshow("Py",py)
cv2.waitKey()
cv2.destroyAllWindows()


In [6]:
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lower = np.array([90, 100, 20])
    higher = np.array([125, 250 ,255])
    
    mask = cv2.inRange(hsv , lower, higher)
    mask = cv2.erode(mask , None, iterations=2)
    cnts,_ = cv2.findContours(mask, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(cnts) > 0:
        
        c= max(cnts , key=cv2.contourArea)
        hull = cv2.convexHull(c)
        cv2.drawContours(frame, [hull], -1, (255,0,0), 3)
        cv2.drawContours(frame, c, -1, (0,255,0), 3)
        
        cv2.imshow("Video",frame)
        cv2.imshow("Video1",mask)

        #showing image in new window with name IMG
    if cv2.waitKey(1) & 0xFF==ord('q'):
        #waitkey has 1 as a frame sld stop after 1 sec andthen checks if q is pressed to stop video
        break
        
cv2.destroyAllWindows()#after 
cap.release()              

In [7]:

cv2.destroyAllWindows()#after 
cap.release() 

In [8]:
import math
def calculateAngle(far, start, end):
    """Cosine rule"""
    a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
    b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
    c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
    angle = math.acos((b**2 + c**2 - a**2) / (2*b*c))
    return angle
#This one is going to be of use....
def countFingers(contour):
    hull = cv2.convexHull(contour, returnPoints=False)
    if len(hull) > 3:
        defects = cv2.convexityDefects(contour, hull)
        cnt = 0
        if type(defects) != type(None):
            for i in range(defects.shape[0]):
                #Calculating the angle form the defects
                s, e, f, d = defects[i, 0]
                start = tuple(contour[s, 0])
                end = tuple(contour[e, 0])
                far = tuple(contour[f, 0])
                angle = calculateAngle(far, start, end)
                # Ignore the defects which are small and wide
                # Probably not fingers
                if d > 10000 and angle <= math.pi/2:
                    cnt += 1
        return (True, cnt)
    return (False, 0)

In [9]:
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    roi = frame
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lower = np.array([90, 100, 20])
    higher = np.array([125, 250 ,255])
    
    mask = cv2.inRange(hsv , lower, higher)
    mask = cv2.erode(mask , None, iterations=2)
    cnts,_ = cv2.findContours(mask, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(cnts) > 0:
        
        c= max(cnts , key=cv2.contourArea)
        hull = cv2.convexHull(c)
        cv2.drawContours(roi,[hull],-1,(255,0,0),3)
        count = countFingers(c)
        
        if count[1] < 2:
            if count[1] == 0:
                pyautogui.press('space')
        #cv2.drawContours(frame, [hull], -1, (255,0,0), 3)
        #cv2.drawContours(frame, c, -1, (0,255,0), 3)
        
        cv2.imshow("Video",frame)
        cv2.imshow("Video1",mask)

        #showing image in new window with name IMG
    if cv2.waitKey(1) & 0xFF==ord('q'):
        #waitkey has 1 as a frame sld stop after 1 sec andthen checks if q is pressed to stop video
        break
        
cv2.destroyAllWindows()#after 
cap.release()              

In [10]:
cap = cv2.VideoCapture(0)
fgbg=cv2.createBackgroundSubtractorKNN()
while True:
    _, frame = cap.read()
    mask = fgbg.apply(frame)
    cv2.imshow("Ori",mask)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        #waitkey has 1 as a frame sld stop after 1 sec andthen checks if q is pressed to stop video
        break
        
cv2.destroyAllWindows()#after 
cap.release()   


In [13]:
cap = cv2.VideoCapture(0)
_, first_frame = cap.read() # [B B B]
fgbg=cv2.createBackgroundSubtractorKNN()
while True: #B=background and A=object
    _, frame = cap.read() #[ B A B ]
    mask = fgbg.apply(frame) #[1 0 1] ---MAsking 
    mask = cv2.dilate(mask , None)
    replace = cv2.bitwise_and(first_frame,first_frame,mask = mask) # [0  B 0]
    mask = cv2.bitwise_not(mask)  #[1 0 1]
    frame = cv2.bitwise_and(frame , frame, mask = mask) # [B 0 B]
    frame = cv2.add(frame,replace) # [B B B]
    cv2.imshow("Video",frame)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        #waitkey has 1 as a frame sld stop after 1 sec andthen checks if q is pressed to stop video
        break
        
cv2.destroyAllWindows()#after 
cap.release()   


In [25]:
face = cv2.CascadeClassifier('C:/Users/DELL/Desktop/CV2_course_material-20190822T085446Z-001/CV2_course_material/haarcascade_frontalface_default.xml')
eye = cv2.CascadeClassifier('C:/Users/DELL/Desktop/CV2_course_material-20190822T085446Z-001/CV2_course_material/haarcascade_eye.xml')
smile = cv2.CascadeClassifier('C:/Users/DELL/Desktop/CV2_course_material-20190822T085446Z-001/CV2_course_material/haarcascade_smile.xml')
cap=cv2.VideoCapture(0)#if 0=inbuilt cam 1=usb web cam
while True:
    _,frame=cap.read()#true false val ret,img
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face.detectMultiScale(gray,1.1,3)
    
    for x,y,w,h in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)
        face_roi= frame[y:y+h,x:x+w]
        gray_roi= frame[y:y+h,x:x+w]
        eyes = eye.detectMultiScale(gray_roi,1.1,3)
        smiles = smile.detectMultiScale(gray,1.5,5)
        for x1,y1,w1,h1 in eyes:
                            cv2.rectangle(face_roi,(x1,y1),(x1+w1,y1+h1),(0,0,255),3)
                            
    cv2.imshow("",frame)#showing image in new window with name IMG
    if cv2.waitKey(1) & 0xFF==ord('q'):
        #waitkey has 1 as a frame sld stop after 1 sec andthen checks if q is pressed to stop video
        break
        
cv2.destroyAllWindows()#after 
cap.release()    

In [ ]:
}